# Optimization in Deep Learning
---
Optimization algorithm which could help us minimize an objective function known as loss function have always been a crucial field of study in machine learning research. This notebook is going to summarize several common optimization algorithms such as SGD, Momentum, Nesterov Momentum, AdaGrad, RMSProp, Adam and also will include more advanced optimization thoughts in the future.

Let's first try to use normal gradient descent optimizer to train a common CNN MNIST classifier.

In [1]:
import numpy as np
import tensorflow as tf

/Users/changlongjiang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [26]:
mnist.train.images.shape

(55000, 784)

In [15]:
# donwload MNIST dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=False)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [16]:
def cnn_model_fn(features, labels, mode):
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [17]:
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

In [19]:
# Create the Estimator
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c2e677860>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [20]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

In [ ]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data[:1000]},
    y=train_labels[:1000],
    batch_size=100,
    num_epochs=None,
    shuffle=True)
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=20000,
    hooks=[logging_hook])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 20001 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:probabilities = [[0.9990689  0.         0.00084926 0.00001535 0.00000004 0.00004459
  0.00000312 0.00000223 0.0000002  0.00001639]
 [0.00038699 0.00000246 0.00001877 0.00110627 0.20147471 0.00020872
  0.00002327 0.00148685 0.0003059  0.794986  ]
 [0.00000037 0.00000001 0.00000077 0.00004274 0.00001347 0.00004683
  0.         0.9989274  0.00000171 0.00096669]
 [0.999966   0.         0.00000909 0.00000034 0.         0.00001595
  0.0000007  0.0000023  0.00000016 0.00000548]
 [0.00000494 0.00000071 0.00003841 0.00006748 0.00000042 0.00000064
  0.         0.99949837 0.000118

INFO:tensorflow:loss = 0.08713771, step = 20001
INFO:tensorflow:probabilities = [[0.00002794 0.00000592 0.00179656 0.00388219 0.02157231 0.00004008
  0.00000627 0.02865781 0.01934867 0.92466223]
 [0.00000216 0.00003198 0.00322691 0.01130251 0.0022386  0.0000168
  0.00000072 0.9580898  0.00125422 0.02383629]
 [0.999959   0.         0.00001362 0.00001365 0.         0.00000486
  0.00000004 0.00000863 0.00000002 0.00000012]
 [0.00000003 0.00000015 0.9999733  0.00002532 0.         0.00000002
  0.00000002 0.00000005 0.00000122 0.00000001]
 [0.00000269 0.00000004 0.00000214 0.00000002 0.00000681 0.00000956
  0.99997747 0.         0.00000113 0.00000005]
 [0.00000001 0.         0.00000022 0.00000017 0.         0.00000004
  0.         0.99999917 0.00000002 0.00000041]
 [0.00000868 0.00000051 0.00000326 0.00005295 0.01085736 0.00024841
  0.00000007 0.07323789 0.00130894 0.9142819 ]
 [0.00000244 0.00000513 0.00004084 0.00003843 0.99881965 0.00084997
  0.00007476 0.00001684 0.00006406 0.00008788]
 

INFO:tensorflow:global_step/sec: 7.19759
INFO:tensorflow:probabilities = [[0.00000608 0.9946508  0.00053791 0.0009188  0.00003557 0.00004846
  0.00013171 0.00109405 0.00231511 0.0002614 ]
 [0.8615736  0.00002127 0.04951923 0.07939099 0.00000238 0.00682414
  0.00021657 0.00000409 0.00215013 0.00029764]
 [0.00004387 0.00000222 0.0000203  0.00000831 0.00000407 0.00008499
  0.00002163 0.00000032 0.99969304 0.00012134]
 [0.00000106 0.00000324 0.9995758  0.00039982 0.00000001 0.00000007
  0.00000019 0.00000005 0.00001909 0.0000006 ]
 [0.00000013 0.00000053 0.00002567 0.0000842  0.98647296 0.00000877
  0.00000808 0.001139   0.00019711 0.01206348]
 [0.00000449 0.00000003 0.00146653 0.98460567 0.00000004 0.00038364
  0.         0.00002028 0.00698569 0.00653358]
 [0.0000441  0.98587453 0.00289381 0.0001881  0.00004794 0.00002213
  0.0008125  0.00009666 0.00996438 0.00005594]
 [0.00000026 0.00000006 0.00005829 0.00001415 0.00000136 0.00003848
  0.00000221 0.00000001 0.99988306 0.00000213]
 [0.000

INFO:tensorflow:loss = 0.07231412, step = 20101 (13.896 sec)
INFO:tensorflow:probabilities = [[0.00053756 0.000181   0.14026044 0.00012127 0.6342392  0.00341946
  0.21527137 0.00004216 0.00115249 0.00477505]
 [0.00002958 0.00000252 0.00001386 0.00016014 0.05598243 0.00010737
  0.00002559 0.00041973 0.00170734 0.9415515 ]
 [0.00000005 0.00000012 0.9988883  0.0010928  0.00000503 0.00000066
  0.00000002 0.00000054 0.00000389 0.00000865]
 [0.00000799 0.0000002  0.00002133 0.00005359 0.99818665 0.00070402
  0.00002895 0.00002103 0.00000691 0.00096937]
 [0.00000082 0.00000004 0.00020773 0.00006352 0.00000009 0.0000193
  0.         0.99968255 0.00001635 0.00000963]
 [0.00005295 0.9979081  0.00108534 0.00003784 0.0000195  0.00001953
  0.00001377 0.00010689 0.00066501 0.00009106]
 [0.0000803  0.00010139 0.00025207 0.00021071 0.07731223 0.00003712
  0.0000058  0.01690027 0.00303    0.9020701 ]
 [0.00000254 0.00006544 0.00057126 0.00815735 0.01268542 0.00038888
  0.00000405 0.01097107 0.00340471 

INFO:tensorflow:global_step/sec: 7.25803
INFO:tensorflow:probabilities = [[0.0005332  0.00023338 0.00010513 0.00015104 0.00000212 0.9546661
  0.00011813 0.0000872  0.04408078 0.00002295]
 [0.00004741 0.8229671  0.00881606 0.02474026 0.00015261 0.00139422
  0.00030397 0.00548188 0.12971632 0.00638019]
 [0.00347915 0.00000295 0.00028412 0.00019128 0.00002251 0.00209535
  0.9934909  0.00000059 0.00042541 0.00000781]
 [0.00003646 0.00000246 0.00003896 0.03090522 0.00001554 0.96156925
  0.00009046 0.00000314 0.00115091 0.00618752]
 [0.00012605 0.00000007 0.00009338 0.00010955 0.00135867 0.0007876
  0.0000016  0.03321723 0.01048007 0.95382583]
 [0.0000112  0.00002398 0.00030565 0.0000572  0.00012932 0.00124494
  0.98691076 0.00000029 0.01130889 0.00000784]
 [0.00016393 0.98922324 0.00084601 0.00044666 0.00011453 0.00003216
  0.00001976 0.00809302 0.00063932 0.00042139]
 [0.00000359 0.00000217 0.00027425 0.9995472  0.00000003 0.00001998
  0.00000003 0.00000005 0.00014696 0.00000578]
 [0.00009

...

INFO:tensorflow:loss = 0.008406876, step = 31401 (15.377 sec)
INFO:tensorflow:probabilities = [[0.0000014  0.         0.00000022 0.00000024 0.99989605 0.00000037
  0.00000004 0.00000265 0.00000004 0.00009906]
 [0.9999938  0.00000001 0.0000049  0.00000001 0.         0.00000053
  0.00000055 0.         0.         0.00000007]
 [0.00226728 0.00001078 0.00002383 0.99346876 0.00000002 0.00341822
  0.         0.00000004 0.00047858 0.00033245]
 [0.00000078 0.         0.00000118 0.99998474 0.         0.00000091
  0.         0.00000014 0.         0.00001231]
 [0.         0.00000013 0.00000001 0.00001309 0.9995926  0.00000469
  0.00000178 0.00000001 0.0002188  0.00016897]
 [0.9503712  0.00004373 0.00026528 0.00137479 0.00000629 0.00186255
  0.04488861 0.0010953  0.0000015  0.00009088]
 [0.00000054 0.00000015 0.00217948 0.00004809 0.00000031 0.00000011
  0.         0.99655956 0.00109012 0.00012165]
 [0.00000013 0.9991685  0.0000102  0.00054089 0.0000004  0.00000021
  0.         0.00000472 0.0001836

INFO:tensorflow:global_step/sec: 6.85789
INFO:tensorflow:probabilities = [[0.00000022 0.00000488 0.00085711 0.9360722  0.00000031 0.00002838
  0.00000066 0.00004727 0.06272207 0.00026683]
 [0.0000001  0.         0.00000034 0.00000921 0.00001881 0.00000076
  0.         0.00537197 0.00000083 0.9945979 ]
 [0.00385494 0.00080246 0.00655358 0.00166265 0.98313415 0.00164878
  0.00164869 0.00061475 0.00006902 0.00001108]
 [0.00018498 0.00076807 0.01230221 0.9781979  0.00125676 0.00647997
  0.00011298 0.00000011 0.00045035 0.00024679]
 [0.00000016 0.         0.00000016 0.00055285 0.00000018 0.99905664
  0.00000004 0.00000108 0.00000198 0.00038699]
 [0.00000064 0.00000279 0.00009421 0.00483776 0.00007817 0.0000005
  0.         0.99122286 0.00000201 0.00376113]
 [0.00000167 0.00004982 0.9975866  0.00016059 0.00000001 0.00000027
  0.00000003 0.00096998 0.00123086 0.00000024]
 [0.99999094 0.00000001 0.00000621 0.00000005 0.         0.00000031
  0.00000102 0.00000118 0.00000002 0.00000027]
 [0.0067

INFO:tensorflow:loss = 0.03658616, step = 31501 (14.579 sec)
INFO:tensorflow:probabilities = [[0.00000032 0.9999151  0.00001801 0.00000309 0.00000007 0.00000028
  0.00000093 0.00000104 0.00006074 0.0000004 ]
 [0.         0.00000091 0.00000025 0.99998224 0.         0.00001093
  0.         0.         0.00000153 0.00000423]
 [0.9999058  0.         0.0000106  0.00000013 0.         0.00000881
  0.00007377 0.00000002 0.00000002 0.00000084]
 [0.00000406 0.00032397 0.9775489  0.01870622 0.00011547 0.00002143
  0.00161957 0.00143652 0.00020133 0.00002248]
 [0.00000007 0.         0.00000034 0.         0.00000115 0.00000045
  0.999998   0.         0.         0.        ]
 [0.99952316 0.         0.00000131 0.         0.00000001 0.
  0.00047553 0.         0.         0.        ]
 [0.00001296 0.00118151 0.00000254 0.02488426 0.00000282 0.9733556
  0.0000008  0.00000091 0.00031005 0.00024848]
 [0.00000012 0.00000661 0.00000014 0.00034061 0.00046374 0.00000265
  0.         0.00011997 0.00061382 0.998452

INFO:tensorflow:global_step/sec: 6.69016
INFO:tensorflow:probabilities = [[0.00023462 0.         0.0000238  0.00000066 0.00000201 0.00000048
  0.         0.00013768 0.00001259 0.9995881 ]
 [0.00000102 0.00000001 0.00097276 0.00000303 0.00000757 0.00000196
  0.99901354 0.         0.00000015 0.        ]
 [0.00402366 0.00000001 0.00078169 0.00000013 0.0000018  0.00000614
  0.00000414 0.00004512 0.00000238 0.9951349 ]
 [0.00000005 0.00000805 0.0000001  0.00063832 0.00010016 0.00007768
  0.00000001 0.00000773 0.00256285 0.99660504]
 [0.00000027 0.0000901  0.00094317 0.00014591 0.00007609 0.00014289
  0.99788886 0.         0.00071264 0.00000003]
 [0.9999999  0.         0.00000011 0.         0.         0.
  0.         0.00000003 0.         0.00000002]
 [0.00000498 0.00000687 0.00000113 0.00000024 0.00000007 0.00000078
  0.00000002 0.         0.9999771  0.00000886]
 [0.00000384 0.99785906 0.00140555 0.00010172 0.0001745  0.00001232
  0.00001071 0.00003298 0.00039814 0.00000125]
 [0.00012851 0.

INFO:tensorflow:loss = 0.009806005, step = 31601 (14.949 sec)
INFO:tensorflow:probabilities = [[0.13281004 0.         0.00001013 0.00081416 0.00000003 0.00000155
  0.00042566 0.0000123  0.00249731 0.86342883]
 [0.00001299 0.00000624 0.00000213 0.16713819 0.00000105 0.8277911
  0.00000322 0.0000003  0.00243622 0.00260852]
 [0.         0.00000001 0.9999981  0.0000012  0.00000029 0.00000001
  0.0000002  0.         0.00000029 0.        ]
 [0.00000743 0.         0.99113667 0.00883595 0.         0.
  0.00000002 0.0000123  0.00000769 0.00000004]
 [0.00000006 0.00000272 0.0000049  0.999833   0.         0.00012034
  0.         0.00000001 0.00001121 0.00002774]
 [0.00000002 0.         0.00000002 0.00000014 0.         0.00001125
  0.00000001 0.         0.99998856 0.        ]
 [0.00001946 0.         0.00000002 0.00000053 0.00000036 0.0000014
  0.         0.993218   0.00000002 0.00676014]
 [0.9997055  0.00000025 0.00023671 0.0000104  0.00000005 0.00000061
  0.00000331 0.00000047 0.00003785 0.000004

In [22]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-04-00:07:47
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-04-00:07:51
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.9691, global_step = 20000, loss = 0.10681324
{'accuracy': 0.9691, 'loss': 0.10681324, 'global_step': 20000}


In [31]:
eval_results

{'accuracy': 0.9691, 'global_step': 20000, 'loss': 0.10681324}

## 1. Stochastic Gradient Descent
Optimization problem:

Minimize $ L(w) = \sum_{i=1}^{l} \mathcal{L}(w;x_{i};y_i)$

$w^{0}--->initialization$

While True:

<pre>i = random index between 1 and l</pre>
    
$\space\space \space\space\space\space\space   w^t = w^{t-1} - y_t\triangledown \mathcal{L}(w^{t-1};x_{i};y_i)$
    
$\space\space \space\space\space\space\space if \space\space\|w^t - w^{t-1}\| < \varepsilon,\space then\space break$

#### Summary:
- Noisy updates lead to fluctuations
- Needs only one example on each step
- Choosing learning rate could be very difficult

## 2. Mini-batch Stochastic Gradient Descent
Optimization problem:

Minimize $ L(w) = \sum_{i=1}^{l} \mathcal{L}(w;x_{i};y_i)$

$w^{0}--->initialization$

While True:

$\space\space \space\space\space\space\space   i_1,i_2,...,i_m = $random indices between 1 and l
    
$\space\space \space\space\space\space\space   w^t = w^{t-1} - y_t \frac{1}{m} \sum _{j=1}^{m}\triangledown \mathcal{L}(w^{t-1};x_{ij};y_{ij})$
    
$\space\space \space\space\space\space\space if \space\space\|w^t - w^{t-1}\| < \varepsilon,\space then\space break$

#### Summary:
- Reduces the variance of gradient approximations
- choosing learning rate is still a problem

## 3. Momentum
$$ $$